DATASET

In [ ]:
#!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
!wget https://github.com/Diegojfsr/RNA_Trabalho_Pratico_II/blob/main/flowers.zip


In [ ]:
### descompactar o arquivo Zipado

#!unzip cats_and_dogs_filtered.zip
!unzip flowers.zip

In [ ]:
### escluir o arquivo zipado

#!rm -rf cats_and_dogs_filtered.zip
!rm -rf flowers.zip

In [ ]:
!pip install tensorflow

In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
#dataset_dir = os.path.join(os.getcwd(), 'cats_and_dogs_filtered')

#dataset_train_dir = os.path.join(dataset_dir, 'train')
#dataset_train_cats_len = len(os.listdir(os.path.join(dataset_train_dir, 'cats')))
#dataset_train_dogs_len = len(os.listdir(os.path.join(dataset_train_dir, 'dogs')))

#dataset_validation_dir = os.path.join(dataset_dir, 'validation')
#dataset_validation_cats_len = len(os.listdir(os.path.join(dataset_validation_dir, 'cats')))
#dataset_validation_dogs_len = len(os.listdir(os.path.join(dataset_validation_dir, 'dogs')))

#print('Train Cats: %s' % dataset_train_cats_len)
#print('Train Dogs: %s' % dataset_train_dogs_len)
#print('Validation Cats: %s' % dataset_validation_cats_len)
#print('Validation Dogs: %s' % dataset_validation_dogs_len)

In [ ]:
#### Importando as Imagens
### Sepando em Train and test
## Verificando a quantidade de imagens

dataset = os.path.join(os.getcwd(), 'flowers')

dataset_train = os.path.join(dataset, 'train')
dataset_train_rose = len(os.listdir(os.path.join(dataset_train, 'rose')))
dataset_train_tulip = len(os.listdir(os.path.join(dataset_train, 'tulip')))

dataset_test = os.path.join(dataset, 'test')
dataset_test_rose = len(os.listdir(os.path.join(dataset_test, 'rose')))
dataset_test_tulip = len(os.listdir(os.path.join(dataset_test, 'tulip')))

print('Train rose: %s' % dataset_train_rose)
print('Train tulip: %s' % dataset_train_tulip)
print('Test rose: %s' % dataset_test_rose)
print('Test tulip: %s' % dataset_test_tulip)

Train rose: 634
Train tulip: 794
Test rose: 150
Test tulip: 190


In [ ]:
#image_width = 160
#image_height = 160
#image_color_channel = 3
#image_color_channel_size = 255
#image_size = (image_width, image_height)
#image_shape = image_size + (image_color_channel,)

#batch_size = 32
#epochs = 20
#learning_rate = 0.0001

#class_names = ['cat', 'dog']

Pre-Processamento

In [ ]:
image_width = 64
image_height = 64
image_color_channel = 3
image_color_channel_size = 255
image_size = (image_width, image_height)
image_shape = image_size + (image_color_channel,)

batch_size = 32  #quantidade de features trazidas por vez do dataset
epochs = 20 #quantidade de X que sera passado pela rede
learning_rate = 0.0001 #taxa de aprendizagem

class_names = ['rose', 'tulip'] #saida

In [ ]:
dataset_train = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_train,
    image_size = image_size,
    batch_size = batch_size,
    shuffle = True
)

Found 1428 files belonging to 2 classes.


In [ ]:
dataset_test = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_test,
    image_size = (image_width, image_height),
    batch_size = batch_size,
    shuffle = True
)

Found 340 files belonging to 2 classes.


In [ ]:
#dataset_validation_cardinality = tf.data.experimental.cardinality(dataset_validation)
#dataset_validation_batches = dataset_validation_cardinality // 5

#dataset_test = dataset_validation.take(dataset_validation_batches)
#dataset_validation = dataset_validation.skip(dataset_validation_batches)

#print('Validation Dataset Cardinality: %d' % tf.data.experimental.cardinality(dataset_validation))
#print('Test Dataset Cardinality: %d' % tf.data.experimental.cardinality(dataset_test))

In [ ]:
dataset_test_cardinality = tf.data.experimental.cardinality(dataset_test)
dataset_test_batches = dataset_test_cardinality // 5

dataset_test2 = dataset_test.take(dataset_test_batches)
dataset_validation = dataset_test.skip(dataset_test_batches)

print('Test Dataset Cardinality: %d' % tf.data.experimental.cardinality(dataset_test))
print('Test2 Dataset Cardinality: %d' % tf.data.experimental.cardinality(dataset_test2))

Test Dataset Cardinality: 11
Test2 Dataset Cardinality: 2


In [ ]:
#autotune = tf.data.AUTOTUNE

#dataset_train = dataset_train.prefetch(buffer_size = autotune)
#dataset_validation = dataset_validation.prefetch(buffer_size = autotune)
#dataset_test = dataset_validation.prefetch(buffer_size = autotune)

In [ ]:

autotune = tf.data.AUTOTUNE

dataset_train = dataset_train.prefetch(buffer_size = autotune)
dataset_test = dataset_test.prefetch(buffer_size = autotune)
dataset_test2 = dataset_test.prefetch(buffer_size = autotune)

In [ ]:
###Função para printar Dataset

def plot_dataset(dataset):

    plt.gcf().clear()
    plt.figure(figsize = (15, 15))

    for features, labels in dataset.take(1):

        for i in range(9):

            plt.subplot(3, 3, i + 1)
            plt.axis('off')

            plt.imshow(features[i].numpy().astype('uint8'))
            plt.title(class_names[labels[i]])

In [ ]:
plot_dataset(dataset_train)

In [ ]:
plot_dataset(dataset_test)

In [ ]:
plot_dataset(dataset_test2)

Criando o Modelo

In [ ]:
data_augmentation = tf.keras.models.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.2)
])

In [ ]:
###Função para Inverter posição das imagens/criando varias imagens apartir de uma

def plot_dataset_data_augmentation(dataset):

    plt.gcf().clear()
    plt.figure(figsize = (15, 15))

    for features, _ in dataset.take(1):

        feature = features[0]

        for i in range(9):

            feature_data_augmentation = data_augmentation(tf.expand_dims(feature, 0))

            plt.subplot(3, 3, i + 1)
            plt.axis('off')

            plt.imshow(feature_data_augmentation[0] / image_color_channel_size)

In [ ]:
plot_dataset_data_augmentation(dataset_train)

Classificação de Camadas

In [ ]:
rescaling = tf.keras.layers.experimental.preprocessing.Rescaling(1. / (image_color_channel_size / 2.), offset = -1, input_shape = image_shape)

Aprendizagem

In [ ]:
model_transfer_learning = tf.keras.applications.MobileNetV2(input_shape = image_shape, include_top = False, weights = 'imagenet')
model_transfer_learning.trainable = False

model_transfer_learning.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)

Modelo

In [ ]:
model = tf.keras.models.Sequential([
    rescaling,
    data_augmentation,
    model_transfer_learning,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
    loss = tf.keras.losses.BinaryCrossentropy(),
    metrics = ['accuracy']
)

model.summary()

In [ ]:
history = model.fit(
    dataset_train,
    validation_data = dataset_test,
    epochs = epochs,
    callbacks = [
        early_stopping
    ]
)

In [ ]:
def plot_model():

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    plt.gcf().clear()
    plt.figure(figsize = (15, 8))

    plt.subplot(1, 2, 1)
    plt.title('Training and Validation Accuracy')
    plt.plot(epochs_range, accuracy, label = 'Training Accuracy')
    plt.plot(epochs_range, val_accuracy, label = 'Validation Accuracy')
    plt.legend(loc = 'lower right')

    plt.subplot(1, 2, 2)
    plt.title('Training and Validation Loss')
    plt.plot(epochs_range, loss, label = 'Training Loss')
    plt.plot(epochs_range, val_loss, label = 'Validation Loss')
    plt.legend(loc = 'lower right')

    plt.show()

In [ ]:
plot_model()

Avaliando o Modelo

In [ ]:
dataset_test_loss, dataset_test_accuracy = model.evaluate(dataset_test)

print('Dataset Test Loss:     %s' % dataset_test_loss)
print('Dataset Test Accuracy: %s' % dataset_test_accuracy)

In [ ]:
def plot_dataset_predictions(dataset):

    features, labels = dataset_test.as_numpy_iterator().next()

    predictions = model.predict_on_batch(features).flatten()
    predictions = tf.where(predictions < 0.5, 0, 1)

    print('Labels:      %s' % labels)
    print('Predictions: %s' % predictions.numpy())

    plt.gcf().clear()
    plt.figure(figsize = (15, 15))

    for i in range(9):

        plt.subplot(3, 3, i + 1)
        plt.axis('off')

        plt.imshow(features[i].astype('uint8'))
        plt.title(class_names[predictions[i]])

In [ ]:
plot_dataset_predictions(dataset_test)

SALVAR E CARREGAR

In [ ]:
model.save('model')

In [ ]:
model = tf.keras.models.load_model('model')

PREDICTIONS

In [ ]:
def predict(image_file):

    image = tf.keras.preprocessing.image.load_img(image_file, target_size = image_size)
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = tf.expand_dims(image, 0)

    prediction = model.predict(image)[0][0]

    print('Prediction: {0} | {1}'.format(prediction, ('rose' if prediction < 0.5 else 'tulip')))

In [ ]:
def predict_url(image_fname, image_origin):

    image_file = tf.keras.utils.get_file(image_fname, origin = image_origin)
    return predict(image_file)

In [ ]:
predict('chico.png')

In [ ]:
predict_url('dog', 'https://www.sciencemag.org/sites/default/files/styles/article_main_large/public/main_puppies_1280p.jpg')